In [ ]:
using PyPlot

In [ ]:
using HDF5

In [ ]:
h5 = h5open("FokkerPlanck_dt1E-6_nt10000.h5", "r")
#h5 = h5open("FokkerPlanck_exp_cos_dt1E-6_nt10.h5", "r")
#h5 = h5open("FokkerPlanck_sin4x_dt1E-6_nt10.h5", "r")
#h5 = h5open("FokkerPlanck_sinx4_dt1E-6_nt10.h5", "r")

In [ ]:
nplot = size(h5["ω"],3)

In [ ]:
ω₀ = view(h5["ω"][:,:,1],     :, :, 1)
ω₁ = view(h5["ω"][:,:,nplot], :, :, 1)
ϕ₀ = view(h5["ϕ"][:,:,1],     :, :, 1)
ϕ₁ = view(h5["ϕ"][:,:,nplot], :, :, 1)
;

In [ ]:
h = zeros(nplot)
for n in 1:nplot
    h[n] = sum_kbn(h5["ω"][:,:,n] .* h5["ϕ"][:,:,n])
end

In [ ]:
ℰ = zeros(nplot)
for n in 1:nplot
    ℰ[n] = sum_kbn(h5["ω"][:,:,n] .* h5["ω"][:,:,n])
end

In [ ]:
figure(figsize=(12,5))

subplot(121)
contour(ω₀, colors="k")
pcolormesh(ω₀)
colorbar()
title("Initial Condition")

subplot(122)
contour(ω₁, colors="k")
pcolormesh(ω₁)
colorbar()
title("Final Solution")

savefig("FokkerPlanck_ω.png", dpi=150)

In [ ]:
figure(figsize=(12,5))

subplot(121)
contour(ϕ₀, colors="k")
pcolormesh(ϕ₀)
colorbar()
title("Initial Condition")

subplot(122)
contour(ϕ₁, colors="k")
pcolormesh(ϕ₁)
colorbar()
title("Final Solution")

savefig("FokkerPlanck_ϕ.png", dpi=150)

In [ ]:
figure(figsize=(12,5))

subplot(121)
scatter(ω₀[:], ϕ₀[:])
title("Initial Condition")
xlabel("ω")
ylabel("ϕ")

subplot(122)
scatter(ω₁[:], ϕ₁[:])
title("Final Solution")
xlabel("ω")
ylabel("ϕ")

savefig("FokkerPlanck_scatter.png", dpi=150)

In [ ]:
figure(figsize=(8,4))
plot((h-h[1])/h[1])
xlabel("t")
ylabel("(h-h₀)/h₀")

In [ ]:
figure(figsize=(8,4))
plot((ℰ-ℰ[1])/ℰ[1])
xlabel("t")
ylabel("(ℰ-ℰ₀)/ℰ₀")